In [1]:
!hostname

v030.ib.bridges2.psc.edu


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Dropout
import tensorflow_addons as tfa
from qhoptim.tf import QHAdamOptimizer
from tensorflow.keras import callbacks
import keras_tuner as kt
import os
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import logging

2024-03-14 17:50:02.339282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 17:50:02.533243: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 17:50:03.904602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/packages/cuda/v11.7.1/lib64:/opt/packages/cuda/v11.7.1/nvvm/lib64:/opt/packages/cuda/v11.7.1/extras/CUPTI/lib64:/jet/home/jlin96/.conda/envs/wandbenv/lib/
2024-03-14 17:50:03.904731: W tensorflow/str

In [3]:
data_path = '/ocean/projects/atm200007p/jlin96/nnspreadtesting_2/overfitmepls/training/training_data/'
train_input = np.load(data_path + 'train_input.npy')
train_target = np.load(data_path + 'train_target.npy')
val_input = np.load(data_path + 'val_input.npy')
val_target = np.load(data_path + 'val_target.npy')

In [4]:
!wandb login

wandb: Currently logged in as: jerrylin247365 (cbrain). Use `wandb login --relogin` to force relogin


In [5]:
def build_model(hp:dict):
    alpha = hp["leak"]
    dp_rate = hp["dropout"]
    batch_norm = hp["batch_normalization"]
    model = Sequential()
    hiddenUnits = hp['hidden_units']
    model.add(Dense(units = hiddenUnits, input_dim=175, kernel_initializer='normal'))
    model.add(LeakyReLU(alpha = alpha))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Dropout(dp_rate))
    for i in range(hp["num_layers"]):
        model.add(Dense(units = hiddenUnits, kernel_initializer='normal'))
        model.add(LeakyReLU(alpha = alpha))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dp_rate))
    model.add(Dense(55, kernel_initializer='normal', activation='linear'))
    initial_learning_rate = hp["lr"]
    optimizer = hp["optimizer"]
    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate = initial_learning_rate)
    elif optimizer == "RMSprop":
        optimizer = keras.optimizers.RMSprop(learning_rate = initial_learning_rate)
    elif optimizer == "RAdam":
        optimizer = tfa.optimizers.RectifiedAdam(learning_rate = initial_learning_rate)
    elif optimizer == "QHAdam":
        optimizer = QHAdamOptimizer(learning_rate = initial_learning_rate, nu2=1.0, beta1=0.995, beta2=0.999)
    model.compile(optimizer = optimizer, loss = 'mse', metrics = ["mse"])
    return model

hp = {'leak':0.37343, \
      'dropout':0.042309, \
      'lr':0.0011942, \
      'hidden_units':336, \
      'num_layers':6, \
      'optimizer':"RAdam", \
      'batch_normalization':1}

model = build_model(hp)

wandb.init(
    # set the wandb project where this run will be logged
    project="debug221", 
    # track hyperparameters and run metadata
    config=hp,
    name='workffs'
)

2024-03-14 17:50:32.903753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 17:50:33.438584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14745 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:15:00.0, compute capability: 7.0
wandb: Currently logged in as: jerrylin247365 (cbrain). Use `wandb login --relogin` to force relogin


In [6]:
model.fit(train_input, train_target, validation_data = (val_input, val_target), epochs = 5, batch_size = 5000, callbacks = [WandbMetricsLogger(), WandbModelCheckpoint(("tuning_directory/debug221"))])

Epoch 1/5
449/449 [==============================] - ETA: 0s - loss: 0.5710 - mse: 0.5710INFO:tensorflow:Assets written to: tuning_directory/debug221/assets


wandb: Adding directory to artifact (./tuning_directory/debug221)... Done. 0.0s


449/449 [==============================] - 19s 35ms/step - loss: 0.5710 - mse: 0.5710 - val_loss: 0.5925 - val_mse: 0.5925
Epoch 2/5
449/449 [==============================] - ETA: 0s - loss: 0.4200 - mse: 0.4200INFO:tensorflow:Assets written to: tuning_directory/debug221/assets


wandb: Adding directory to artifact (./tuning_directory/debug221)... Done. 0.0s


449/449 [==============================] - 13s 28ms/step - loss: 0.4200 - mse: 0.4200 - val_loss: 0.5715 - val_mse: 0.5715
Epoch 3/5
449/449 [==============================] - ETA: 0s - loss: 0.3980 - mse: 0.3980INFO:tensorflow:Assets written to: tuning_directory/debug221/assets


...

449/449 [==============================] - 12s 27ms/step - loss: 0.3980 - mse: 0.3980 - val_loss: 0.5621 - val_mse: 0.5621
Epoch 4/5
446/449 [============================>.] - ETA: 0s - loss: 0.3876 - mse: 0.3876INFO:tensorflow:Assets written to: tuning_directory/debug221/assets


wandb: Adding directory to artifact (./tuning_directory/debug221)... Done. 0.0s


449/449 [==============================] - 12s 27ms/step - loss: 0.3875 - mse: 0.3875 - val_loss: 0.5735 - val_mse: 0.5735
Epoch 5/5
448/449 [============================>.] - ETA: 0s - loss: 0.3809 - mse: 0.3809INFO:tensorflow:Assets written to: tuning_directory/debug221/assets


wandb: Adding directory to artifact (./tuning_directory/debug221)... Done. 0.1s


449/449 [==============================] - 12s 28ms/step - loss: 0.3810 - mse: 0.3810 - val_loss: 0.5449 - val_mse: 0.5449


In [7]:
model.save('wtf.h5')